- Name: Isaac Ndirangu Muturi
- Email: ndirangumuturi749@gmail.com

# Homework

## Setting up

In order to get a static set of results, we will use historical data from the dataset.

Run the following commands:
```bash
# Load the cluster op commands.
source commands.sh
# First, reset the cluster:
clean-cluster
# Start a new cluster
start-cluster
# wait for cluster to start
sleep 5
# Seed historical data instead of real-time data
seed-kafka
# Recreate trip data table
psql -f risingwave-sql/table/trip_data.sql
# Wait for a while for the trip_data table to be populated.
sleep 5
# Check that you have 100K records in the trip_data table
# You may rerun it if the count is not 100K
psql -c "SELECT COUNT(*) FROM trip_data"
```


## Question 0

_This question is just a warm-up to introduce dynamic filter, please attempt it before viewing its solution._

What are the dropoff taxi zones at the latest dropoff times?

For this part, we will use the [dynamic filter pattern](https://docs.risingwave.com/docs/current/sql-pattern-dynamic-filters/).

<details>
<summary>Solution</summary>

```sql
CREATE MATERIALIZED VIEW latest_dropoff_time AS
    WITH t AS (
        SELECT MAX(tpep_dropoff_datetime) AS latest_dropoff_time
        FROM trip_data
    )
    SELECT taxi_zone.Zone as taxi_zone, latest_dropoff_time
    FROM t,
            trip_data
    JOIN taxi_zone
        ON trip_data.DOLocationID = taxi_zone.location_id
    WHERE trip_data.tpep_dropoff_datetime = t.latest_dropoff_time;

--    taxi_zone    | latest_dropoff_time
-- ----------------+---------------------
--  Midtown Center | 2022-01-03 17:24:54
-- (1 row)
```

</details>



## Question 1

Create a materialized view to compute the average, min and max trip time **between each taxi zone**.

Note that we consider the do not consider `a->b` and `b->a` as the same trip pair.
So as an example, you would consider the following trip pairs as different pairs:
```plaintext
Yorkville East -> Steinway
Steinway -> Yorkville East
```

From this MV, find the pair of taxi zones with the highest average trip time.
You may need to use the [dynamic filter pattern](https://docs.risingwave.com/docs/current/sql-pattern-dynamic-filters/) for this.

Bonus (no marks): Create an MV which can identify anomalies in the data. For example, if the average trip time between two zones is 1 minute,
but the max trip time is 10 minutes and 20 minutes respectively.

Options:
1. Yorkville East, Steinway
2. Murray Hill, Midwood
3. East Flatbush/Farragut, East Harlem North
4. Midtown Center, University Heights/Morris Heights

p.s. The trip time between taxi zones does not take symmetricity into account, i.e. `A -> B` and `B -> A` are considered different trips. This applies to subsequent questions as well.



In [13]:
# -- Create a materialized view to calculate trip statistics between pickup and dropoff locations
trip_time_statistics="""
CREATE MATERIALIZED VIEW trip_time_statistics AS
SELECT
    taxi_zoneP.zone AS pickup_zone,
    taxi_zoneD.zone AS dropoff_zone,
    AVG(trip_data.tpep_dropoff_datetime - trip_data.tpep_pickup_datetime) AS avg_triptime,
    MIN(trip_data.tpep_dropoff_datetime - trip_data.tpep_pickup_datetime) AS min_triptime,
    MAX(trip_data.tpep_dropoff_datetime - trip_data.tpep_pickup_datetime) AS max_triptime
FROM 
    trip_data
JOIN 
    taxi_zone AS taxi_zoneP ON trip_data.pulocationid = taxi_zoneP.location_id
JOIN 
    taxi_zone AS taxi_zoneD ON trip_data.dolocationid = taxi_zoneD.location_id
GROUP BY 
    taxi_zoneP.zone, taxi_zoneD.zone;
"""

!psql -h localhost -p 4566 -U root -d dev -c "{trip_time_statistics}"

CREATE_MATERIALIZED_VIEW


In [14]:
# -- Step 2: Find the pair of taxi zones with the highest average trip time
max_avg_trip_time_query = """
SELECT
    pickup_zone,
    dropoff_zone,
    avg_triptime
FROM
    trip_time_statistics
ORDER BY
    avg_triptime DESC
LIMIT 1
"""

!psql -h localhost -p 4566 -U root -d dev -c "{max_avg_trip_time_query}"


  pickup_zone   | dropoff_zone | avg_triptime 
----------------+--------------+--------------
 Yorkville East | Steinway     | 23:59:33
(1 row)



In [ ]:
# Answer: Yorkville East, Steinway

## Question 2

Recreate the MV(s) in question 1, to also find the **number of trips** for the pair of taxi zones with the highest average trip time.

Options:
1. 5
2. 3
3. 10
4. 1



In [9]:
# -- Create a materialized view to calculate trip statistics between pickup and dropoff locations
trip_time_statistics_with_count="""
CREATE MATERIALIZED VIEW trip_time_statistics_with_count AS
SELECT
    taxi_zoneP.zone AS pickup_zone,
    taxi_zoneD.zone AS dropoff_zone,
    AVG(trip_data.tpep_dropoff_datetime - trip_data.tpep_pickup_datetime) AS avg_triptime,
    MIN(trip_data.tpep_dropoff_datetime - trip_data.tpep_pickup_datetime) AS min_triptime,
    MAX(trip_data.tpep_dropoff_datetime - trip_data.tpep_pickup_datetime) AS max_triptime,
    COUNT(*) AS num_trips 
FROM 
    trip_data
JOIN 
    taxi_zone AS taxi_zoneP ON trip_data.pulocationid = taxi_zoneP.location_id
JOIN 
    taxi_zone AS taxi_zoneD ON trip_data.dolocationid = taxi_zoneD.location_id
GROUP BY 
    taxi_zoneP.zone, taxi_zoneD.zone;
"""

!psql -h localhost -p 4566 -U root -d dev -c "{trip_time_statistics_with_count}"

CREATE_MATERIALIZED_VIEW


In [11]:
# -- Retrieve records with the maximum average trip time
records_with_max_average_triptime="""
WITH max_average_triptime AS (
    SELECT MAX(avg_triptime) AS max_triptime 
    FROM trip_time_statistics_with_count
)
SELECT 
    tswc.pickup_zone, 
    tswc.dropoff_zone, 
    tswc.avg_triptime, 
    tswc.num_trips 
FROM 
    max_average_triptime AS mat
JOIN 
    trip_time_statistics_with_count AS tswc ON mat.max_triptime = tswc.avg_triptime;
"""

!psql -h localhost -p 4566 -U root -d dev -c "{records_with_max_average_triptime}"

  pickup_zone   | dropoff_zone | avg_triptime | num_trips 
----------------+--------------+--------------+-----------
 Yorkville East | Steinway     | 23:59:33     |         1
(1 row)



In [ ]:
# Answer: 1

## Question 3

From the latest pickup time to 17 hours before, what are the top 3 busiest zones in terms of number of pickups?
For example if the latest pickup time is 2020-01-01 17:00:00,
then the query should return the top 3 busiest zones from 2020-01-01 00:00:00 to 2020-01-01 17:00:00.

HINT: You can use [dynamic filter pattern](https://docs.risingwave.com/docs/current/sql-pattern-dynamic-filters/)
to create a filter condition based on the latest pickup time.

NOTE: For this question `17 hours` was picked to ensure we have enough data to work with.

Options:
1. Clinton East, Upper East Side North, Penn Station
2. LaGuardia Airport, Lincoln Square East, JFK Airport
3. Midtown Center, Upper East Side South, Upper East Side North
4. LaGuardia Airport, Midtown Center, Upper East Side North

In [4]:
busiest_zones_last_17_hrs="""
CREATE MATERIALIZED VIEW busiest_zones_last_17_hrs AS
WITH location_to_location_counts_last_17_hrs AS (
    SELECT
        taxi_zone.zone AS pu_zone,
        COUNT(*) AS num_trips
    FROM
        trip_data
    JOIN 
        taxi_zone ON trip_data.pulocationid = taxi_zone.location_id
    WHERE 
        tpep_pickup_datetime >= (SELECT MAX(tpep_pickup_datetime) - INTERVAL '17 hours' FROM trip_data) 
    GROUP BY 
        taxi_zone.zone
)
SELECT * 
FROM location_to_location_counts_last_17_hrs;
"""

!psql -h localhost -p 4566 -U root -d dev -c "{busiest_zones_last_17_hrs}"

NOTICE:  The ORDER BY clause in the CREATE MATERIALIZED VIEW statement does not guarantee that the rows selected out of this materialized view is returned in this order.
It only indicates the physical clustering of the data, which may improve the performance of queries issued against this materialized view.

CREATE_MATERIALIZED_VIEW


In [8]:
# -- Query the materialized view to retrieve the top 3 pickup zones with the highest number of trips
!psql -h localhost -p 4566 -U root -d dev -c "SELECT * FROM  busiest_zones_last_17_hrs ORDER BY num_trips DESC LIMIT 10;"


           pu_zone            | num_trips 
------------------------------+-----------
 LaGuardia Airport            |        19
 JFK Airport                  |        17
 Lincoln Square East          |        17
 Penn Station/Madison Sq West |        16
 Upper East Side North        |        13
 Times Sq/Theatre District    |        12
 East Chelsea                 |        11
 Upper East Side South        |        10
 Clinton East                 |         9
 Lenox Hill West              |         8
(10 rows)



In [ ]:
# Answer: La Guardia, Lincoln Square East, JFK Airport
